In [30]:
#Import libraries
import pandas as pd
import numpy as np
import math
import statistics as stats
import sys
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

## 2. Cleaning Eurostat application and decision dataframes
 #### A few differences from UNHCR data
 - Annual data since 2008;
 - There is aggregated data per EU27, EU28 and extra communitarian countries; 
 - Greater number of considered countries, such as the Vatican, and they use iso code 2;
 - There is monthly data available, important to compare with Covid-19 cases;
 - There is only data per person, not per case;
 - Includes information on gender and age;
 - Asylum application types includes asylum (person depositing an applications), first application (in a different Member State) and subsequent claims.

### 2.1. Asylum applicants by type of application, citzenship, age, sex - Annual data

In [31]:
euro_applications = pd.read_csv('data/Eurostat/migr_asyappctza_linear.csv')
euro_applications.columns = list(map(lambda x: x.lower().replace(' ', '_'), euro_applications.columns))

In [32]:
euro_applications.columns

Index(['dataflow', 'last_update', 'freq', 'citizen', 'sex', 'unit', 'age',
       'asyl_app', 'geo', 'time_period', 'obs_value', 'obs_flag'],
      dtype='object')

In [33]:
euro_applications.apply(pd.Series.unique)

dataflow                            [ESTAT:MIGR_ASYAPPCTZA(1.0)]
last_update                                  [07/04/22 11:00:00]
freq                                                         [A]
citizen        [AD, AE, AF, AG, AL, AM, AO, AR, AT, AU, AZ, B...
sex                                               [F, M, T, UNK]
unit                                                       [PER]
age            [TOTAL, UNK, Y14-17, Y18-34, Y35-64, Y_GE65, Y...
asyl_app                               [ASY_APP, NASY_APP, SSEQ]
geo            [AT, BE, BG, CH, CY, CZ, DE, DK, EE, EL, ES, E...
time_period    [2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...
obs_value      [0.0, 5.0, 10.0, 15.0, 25.0, 20.0, 410.0, 480....
obs_flag                                             [nan, d, p]
dtype: object

In [34]:
euro_applications = euro_applications.drop(['dataflow', 'last_update', 'freq', 'obs_flag','unit'], axis=1)
#Frequency is annual;
#Unit is per person only;
#Flag footnote
#Drop Total from age
euro_applications = euro_applications[~euro_applications['age'].isin(['TOTAL'])]
euro_applications = euro_applications[~euro_applications['sex'].isin(['T'])]
#Only need new applications
euro_applications = euro_applications[~euro_applications['asyl_app'].isin(['SSEQ','NASY_APP'])]

In [35]:
euro_applications['citizen'].isna().sum()/len(euro_applications['citizen'])
#Very small number of Nans, so we will be placing then together with UNK"
euro_applications['citizen'] = euro_applications['citizen'].fillna('UNK')
euro_applications = euro_applications[~euro_applications['citizen'].isin(['EXT_EU27_2020','EXT_EU28','TOTAL'])]

In [36]:
euro_applications['citizen'].isin(['UNK']).sum()/len(euro_applications['citizen'])

0.0097984363369222

In [37]:
euro_applications.head()

,citizen,sex,age,asyl_app,geo,time_period,obs_value
857,AD,F,UNK,ASY_APP,AT,2008,0.0
858,AD,F,UNK,ASY_APP,AT,2009,0.0
859,AD,F,UNK,ASY_APP,AT,2010,0.0
860,AD,F,UNK,ASY_APP,AT,2011,0.0
861,AD,F,UNK,ASY_APP,AT,2012,0.0


### 2.2. Asylum applicants by citzenship, age, sex - Monthly data - Nov-2019-Mar-2022

euro_mon_app = pd.read_csv('data/Eurostat/migr_asyappctzm_forhypo2_linear.csv')
euro_mon_app.columns = list(map(lambda x: x.lower().replace(' ', '_'), euro_mon_app.columns))

euro_mon_app['time_period'] = pd.to_datetime(euro_mon_app['time_period'], errors = "coerce")
euro_mon_app['time_period'] = euro_mon_app['time_period'][0].strftime(format='%m/%Y')

euro_mon_app = euro_mon_app.drop(['dataflow', 'last_update', 'freq', 'obs_flag','unit'], axis=1)
#Unit is per person only;
 Flag footnote d = definition differs, see metadata 

euro_mon_app = euro_mon_app[~euro_mon_app['citizen'].isin(['EXT_EU27_2020','EXT_EU28','TOTAL'])]

euro_mon_app.apply(pd.Series.unique)

euro_mon_app.head()

euro_mon_app['time_period'].value_counts()

### 2.3. Asylum applicants by citzenship, age, sex - Monthly data - 2018-2021

In [38]:
euro_mon_app2 = pd.read_csv('data/Eurostat/migr_asyappctzm__custom_2596801_linear.csv')
euro_mon_app2.columns = list(map(lambda x: x.lower().replace(' ', '_'), euro_mon_app2.columns))

In [39]:
#euro_mon_app['year'] = euro_mon_app2['time_period'][0].strftime(format='%Y').dt.year
#euro_mon_app2 = euro_mon_app2[euro_mon_app2['year'].isin([2018,2019,2020,2021])]

In [40]:
#euro_mon_app2['time_period'] = pd.to_datetime(euro_mon_app2['time_period'], errors = "coerce")
#euro_mon_app2['time_period'] = euro_mon_app2['time_period'].strftime(format='%m/%Y')

In [41]:
euro_mon_app2 = euro_mon_app2.drop(['dataflow', 'last_update', 'freq', 'obs_flag','unit'], axis=1)
#Unit is per person only;
# Flag footnote d = definition differs, see metadata 

In [42]:
#euro_mon_app = euro_mon_app[~euro_mon_app['citizen'].isin(['EXT_EU27_2020','EXT_EU28','TOTAL'])]

In [43]:
euro_mon_app2.apply(pd.Series.unique)

citizen                                [EXT_EU27_2020, EXT_EU28]
sex                                                          [T]
age                                                      [TOTAL]
asyl_app                                     [ASY_APP, NASY_APP]
geo                                              [EU27_2020, UK]
time_period    [2018-01, 2018-02, 2018-03, 2018-04, 2018-05, ...
obs_value      [56665, 48855, 49990, 50580, 52030, 50605, 541...
dtype: object

In [44]:
euro_mon_app2 = euro_mon_app2[~euro_mon_app2['asyl_app'].isin(['NASY_APP'])]

### 2.4. First instance decisions on asylum applications

In [45]:
euro_decisions = pd.read_csv('data/Eurostat/migr_asydcfsta_linear.csv')
euro_decisions.columns = list(map(lambda x: x.lower().replace(' ', '_'), euro_decisions.columns))

C:\Users\franc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
euro_decisions.shape

(18547796, 12)

In [47]:
euro_decisions = euro_decisions[~euro_decisions['citizen'].isin(['EXT_EU27_2020','EXT_EU28','TOTAL'])]

In [48]:
euro_decisions['decision'].value_counts()

TOTAL        2881284
TOTAL_POS    2881284
REJECTED     2730142
GENCONV      2719340
SUB_PROT     2697965
TEMP_PROT    2344612
HUMSTAT      2033195
Name: decision, dtype: int64

In [49]:
euro_decisions = euro_decisions.drop(['dataflow', 'last_update', 'freq', 'obs_flag', 'unit'], axis=1)
#Frequency is annual;
#Unit is per person only;
# Flag footnote d = definition differs, see metadata 
euro_decisions = euro_decisions[~euro_decisions['age'].isin(['TOTAL'])]
euro_decisions = euro_decisions[~euro_decisions['sex'].isin(['T'])]
euro_decisions = euro_decisions[~euro_decisions['decision'].isin(['TOTAL', 'TEMP_PROT','SUB_PROT','GENCONV','HUMSTAT'])]

In [50]:
euro_decisions.apply(pd.Series.unique)

citizen        [AD, AE, AF, AG, AL, AM, AO, AR, AT, AU, AZ, B...
sex                                                  [F, M, UNK]
age            [UNK, Y14-17, Y18-34, Y35-64, Y_GE65, Y_LT14, ...
decision                                   [REJECTED, TOTAL_POS]
geo            [AT, BE, BG, CH, CY, CZ, DE, DK, EE, EL, ES, E...
time_period    [2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...
obs_value      [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 45.0, 150.0...
dtype: object

In [51]:
euro_decisions.head()

,citizen,sex,age,decision,geo,time_period,obs_value
3656,AD,F,UNK,REJECTED,AT,2008,0.0
3657,AD,F,UNK,REJECTED,AT,2009,0.0
3658,AD,F,UNK,REJECTED,AT,2010,0.0
3659,AD,F,UNK,REJECTED,AT,2011,0.0
3660,AD,F,UNK,REJECTED,AT,2012,0.0


## Creating .csv files

In [52]:
from pathlib import Path  
#Eurostat annual applications
filepath = Path('data/Cleaned/euro_app.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
euro_applications.to_csv(filepath) 

#Eurostat monthly applications Nov-2019-Mar-2022
#filepath = Path('data/Cleaned/euro_mon_app.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
#euro_mon_app.to_csv(filepath) 

#Eurostat monthly applications 2018-2021
filepath = Path('data/Cleaned/euro_mon_app2clean.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
euro_mon_app2.to_csv(filepath) 

#Eurostat annual decisions
filepath = Path('data/Cleaned/euro_dec.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
euro_decisions.to_csv(filepath) 